In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import math
import absl
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
#data = input_data.read_data_sets('data/MNIST/', one_hot=True)
mnist = input_data.read_data_sets('./mnist', one_hot=True)  # they has been normalized to range (0,1)
test_x = mnist.test.images[:10000]
test_y = mnist.test.labels[:10000]

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [3]:
mnist.test.cls = np.argmax(mnist.test.labels, axis=1)

In [4]:
# We know that MNIST images are 28 pixels in each dimension.
img_size = 28

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_size * img_size

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# Number of colour channels for the images: 1 channel for gray-scale.
num_channels = 1

# Number of classes, one class for each of 10 digits.
num_classes = 10

In [5]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x') / 255
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

create the CNN

In [6]:
net = tf.layers.conv2d(inputs=x_image, name='layer_conv1', padding='same',
                       filters=16, kernel_size=5, activation=tf.nn.relu)
net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

# layer_conv2
net = tf.layers.conv2d(inputs=net, name='layer_conv2', padding='same',
                       filters=36, kernel_size=5, activation=tf.nn.relu)
net = tf.layers.max_pooling2d(inputs=net, pool_size=2, strides=2)

print(net)
net = tf.layers.flatten(net)

print(net)
net = tf.layers.dense(inputs=net, name='layer_fc1',
                      units=128, activation=tf.nn.relu)
logits = tf.layers.dense(inputs=net, name='layer_fc_out',
                      units=num_classes, activation=None)
print(logits)

Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 7, 7, 36), dtype=float32)
Tensor("flatten/Reshape:0", shape=(?, 1764), dtype=float32)
Tensor("layer_fc_out/BiasAdd:0", shape=(?, 10), dtype=float32)


/Users/gloria/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
/Users/gloria/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/Users/gloria/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/pooling.py:310: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  warnings.warn('`tf.layers.max_pooling2d` is deprecated and '
/Users/gloria/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core

In [7]:
y_pred = tf.nn.softmax(logits=logits)
y_pred_cls = tf.argmax(y_pred, dimension=1)

In [8]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=logits)
loss = tf.reduce_mean(cross_entropy)

Optimization

In [9]:
opt = tf.train.AdamOptimizer(learning_rate=1e-4)
optimizer = opt.minimize(loss)

Classification Accuracy

In [10]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Getting the weight

In [12]:
trainable_var_list = tf.trainable_variables()
def get_weights_variable(layer_name):
    with tf.variable_scope(layer_name, reuse=True):
        variable = tf.get_variable('kernel')
    return variable

weights_conv1 = get_weights_variable(layer_name='layer_conv1')
weights_conv2 = get_weights_variable(layer_name='layer_conv2')
print(weights_conv1)
print(weights_conv2)

weights_fc1 = get_weights_variable(layer_name='layer_fc1')
weights_fc_out = get_weights_variable(layer_name='layer_fc_out')
print(weights_fc1)
print(weights_fc_out)

<tf.Variable 'layer_conv1/kernel:0' shape=(5, 5, 1, 16) dtype=float32_ref>
<tf.Variable 'layer_conv2/kernel:0' shape=(5, 5, 16, 36) dtype=float32_ref>
<tf.Variable 'layer_fc1/kernel:0' shape=(1764, 128) dtype=float32_ref>
<tf.Variable 'layer_fc_out/kernel:0' shape=(128, 10) dtype=float32_ref>


start one session, and initialize the variables

In [13]:
session = tf.Session()
session.run(tf.global_variables_initializer())

get gradient during training; compute hessian 

In [14]:
#grad = opt.compute_gradients(loss, weights_fc_out)[0]
grads = tf.gradients(loss, weights_fc_out)[0]
print(grads)
hessian = tf.reduce_sum(tf.hessians(loss, weights_fc_out)[0], axis = 2)
print(hessian)
train_batch_size = 64

Tensor("gradients_1/layer_fc_out/MatMul_grad/MatMul_1:0", shape=(128, 10), dtype=float32)
Tensor("Sum:0", shape=(128, 10, 10), dtype=float32)


start the optimization

In [34]:
total_iterations = 0
def optimize(num_iterations):
    global total_iterations

    for i in range(total_iterations,
                   total_iterations + num_iterations):


        x_batch, y_true_batch = mnist.train.next_batch(train_batch_size)

        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}


        session.run(optimizer, feed_dict=feed_dict_train)
        if i % 10 == 0:
            # Calculate the accuracy on the training-set.
            los, acc = session.run([loss, accuracy], feed_dict=feed_dict_train)
            
            grads_vals, hess_vals = session.run([grads, hessian], feed_dict=feed_dict_train)
            print(grads_vals.shape)
            print(hess_vals.shape)

            # Message for printing.
            msg = "Iteration: {0:>6}, Training Loss: {1:>1.6}, Training Accuracy: {2:>6.1%}"

            # Print it.
            print(msg.format(i + 1, los, acc))

    # Update the total number of iterations performed.
    total_iterations += num_iterations

In [18]:
optimize(num_iterations=99)

(128, 10)
(128, 10, 10)
Iteration:      1, Training Loss: 2.29611, Training Accuracy:  20.3%
(128, 10)
(128, 10, 10)
Iteration:     11, Training Loss: 2.22555, Training Accuracy:  29.7%
(128, 10)
(128, 10, 10)
Iteration:     21, Training Loss: 2.17334, Training Accuracy:  29.7%
(128, 10)
(128, 10, 10)
Iteration:     31, Training Loss: 2.16627, Training Accuracy:  43.8%
(128, 10)
(128, 10, 10)
Iteration:     41, Training Loss: 2.03829, Training Accuracy:  68.8%
(128, 10)
(128, 10, 10)
Iteration:     51, Training Loss: 1.99218, Training Accuracy:  64.1%
(128, 10)
(128, 10, 10)
Iteration:     61, Training Loss: 1.75646, Training Accuracy:  73.4%
(128, 10)
(128, 10, 10)
Iteration:     71, Training Loss: 1.53628, Training Accuracy:  84.4%
(128, 10)
(128, 10, 10)
Iteration:     81, Training Loss: 1.32736, Training Accuracy:  87.5%
(128, 10)
(128, 10, 10)
Iteration:     91, Training Loss: 1.35038, Training Accuracy:  67.2%


In [27]:
def plot_conv_weights(weights, input_channel=0):

    w = session.run(weights)

    w_min = np.min(w)
    w_max = np.max(w)

    num_filters = w.shape[3]

    num_grids = math.ceil(math.sqrt(num_filters))

    fig, axes = plt.subplots(num_grids, num_grids)

    for i, ax in enumerate(axes.flat):
        if i<num_filters:

            img = w[:, :, input_channel, i]

            ax.imshow(img, vmin=w_min, vmax=w_max,
                      interpolation='nearest', cmap='seismic')

        ax.set_xticks([])
        ax.set_yticks([])

    plt.show()

In [28]:
from sklearn.decomposition import PCA
def plot_fc_weights(weights_list):

    w_list = session.run(weights_list)
    
    pca = PCA(n_components=2)
    
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title('2 component PCA', fontsize = 20)
    
    for w in w_list:
            
        print(w.shape)

        principalComponents = pca.fit_transform(w)

        ax.scatter(principalComponents[:,0], principalComponents[:,1], label=w.shape, alpha=0.5)

    ax.legend()
    plt.show()

In [39]:
weights_list=[]
for step in range(100):
    if step % 3 == 0:
        weights=session.run(trainable_var_list)
        weights=PCA(weights)
        weights_list.append(weights)

In [40]:
print(weights_list)

[PCA(n_components=[array([[[[-7.02866837e-02,  1.08631827e-01,  4.99007441e-02,
          -4.86401096e-02,  1.26112536e-01, -3.08717322e-02,
           4.23055924e-02, -5.49924783e-02,  8.90975073e-02,
           6.02630228e-02, -9.03390869e-02, -1.13589466e-01,
           1.15379550e-01,  1.19940154e-01,  1.08178295e-01,
           9.53982845e-02]],

        [[ 1.21127687e-01,  9.15692896e-02,  2.99458336e-02,
          -7.25981370e-02, -8.97659659...
       [-0.17888653,  0.13743414,  0.0086686 , ...,  0.16559619,
        -0.16327341, -0.07722243],
       [ 0.16925922,  0.13186021,  0.05882641, ..., -0.06325659,
        -0.097877  ,  0.01333893],
       [-0.10582094,  0.09271786, -0.00425618, ..., -0.17365101,
        -0.13302186,  0.01772023]], dtype=float32),
                  array([-0.00025708,  0.00500724, -0.0016247 , -0.00211708, -0.00095305,
        0.00059295, -0.00223883,  0.00116289, -0.00290741,  0.00169671],
      dtype=float32)]), PCA(n_components=[array([[[[-7.02866837

Github see: https://github.com/Gloriabhsfer/DeeplearningCPS843